In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from IPython.display import display
from PIL import Image
import numpy as np
from keras.preprocessing import image

In [ ]:
classifier = Sequential()
classifier.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=(224,224,3)))
classifier.add(MaxPooling2D(pool_size = (2,2), strides=(2,2)))
classifier.add(Conv2D(64, (5, 5), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())
classifier.add(Dense(10, activation = "relu"))
classifier.add(Dense(8, activation = "softmax"))
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

valid_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(
    "dataset",
    target_size = (224,224),
    color_mode="rgb",
    batch_size = 4,
    shuffle = True,
    class_mode = "categorical")

test_set = test_datagen.flow_from_directory(
    "testset",
    target_size = (224,244),
    color_mode="rgb",
    batch_size = 4,
    shuffle = False,
    class_mode = "categorical")

valid_set = valid_datagen.flow_from_directory(
    "validset",
    target_size = (224,224),
    color_mode="rgb",
    batch_size = 4,
    shuffle = True,
    class_mode = "categorical")

In [ ]:
classifier.fit_generator(
    generator=training_set,
    steps_per_epoch = 800,
    epochs = 10,
    validation_data = valid_set,
    validation_steps = 800)
classifier.save("skin_classifier.h5")

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
test_image = image.load_img("ISIC_0024450.jpg", target_size = (224,224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
decision = np.argmax(result)
print(decision)
print(result)

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.contrib import lite

converter = lite.TocoConverter.from_keras_model_file("skin_classifier.h5");
tflite_model = converter.convert()
open("linear.tflite","wb").write(tflite_model)